In [3]:
import apache_beam as beam
with beam.Pipeline() as pipeline:
    icons = (pipeline| 'Garden plants' >> beam.Create([
                  ('🍓', 'Strawberry'),
                  ('🥕', 'Carrot'),
                  ('🍆', 'Eggplant'),
                  ('🍅', 'Tomato'),
                  ('🥔', 'Potato'),
              ])
             |'keys' >> beam.Keys()
             |beam.Map(print)
            )

🍓
🥕
🍆
🍅
🥔


In [8]:
#Filtering with multiple arguments
import apache_beam as beam
def is_perennial(plant):
    return plant['duration']== 'perennial'
def has_duration(plant, duration):
    return plant['duration'] == duration
with beam.Pipeline() as pipeline:
    perennials =(pipeline|'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'
          },
      ])
               #  |'Filter Perennials' >> beam.Filter(is_perennial)
                # | 'filter Perennials' >>beam.Filter(lambda x: x['duration']=='perennial')
                 |'filter  perennials' >> beam.Filter(has_duration,'annual')
                 |beam.Map(print)
                ) 

{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}


In [17]:
#Filtering with side inputs as singletons
import apache_beam as beam
with beam.Pipeline() as pipeline:
    perennial = pipeline|'Perennial' >>beam.Create(["perennial"])
    
    perennials = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'
          },
      ])
        |"Filter pereennials" >> beam.Filter(lambda plant,
                                            duration:plant['duration'] == duration,
                                             duration = beam.pvalue.AsSingleton(perennial)
                                            )
        |beam.Map(print)
    )
    

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}


In [19]:
#Filtering with side inputs as iterators
import apache_beam as beam
with beam.Pipeline() as pipeline:
    valid_durations = pipeline | 'valid durations' >> beam.Create([
        'annual','biennial','perennial'        
    ])
    
    valid_plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'PERENNIAL'
          },
      ])
        |'Filter valid plants' >> beam.Filter(
        lambda plant,
            valid_durations: plant['duration'] in valid_durations,
            valid_durations = beam.pvalue.AsIter(valid_durations)
        )
        |beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}


In [32]:
#Filtering with side inputs as dictionaries
import apache_beam as beam

with beam.Pipeline() as pipeline:
    keep_duration = pipeline| 'duation filters' >> beam.Create([
        ('annual', False),
      ('biennial', False),
      ('perennial', True),
    ])


    perrennials = (
    pipeline
      | 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'
          },
      ])
        |'Filter plants by duration' >>beam.Filter(
        lambda plant,
            keep_duration:keep_duration[plant['duration']],
            keep_duration = beam.pvalue.AsDict(keep_duration)
        )
        |beam.Map(print)
    )

    #print(keep_duration)

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}
